In [ ]:
## Mosaic Image Generator

This Python script generates a mosaic image from a pool of images by matching small sections of a target input image with the most similar images from the pool. It uses the average color of images to find the best matches and constructs a larger mosaic image.

### Requirements
- **Python Packages:** Make sure to install the required Python packages listed in `requirements.txt` using the following command:


In [ ]:

### File and Folder Structure
- **data/input.jpg:** This is the target image for which the mosaic will be generated. Update the `input_path` in the `Options` class if your target image is located elsewhere or has a different name.
- **data/output.jpg:** This is the path where the resulting mosaic image will be saved. Update the `output_path` in the `Options` class if you wish to change the location or name of the output file.
- **jds:** This folder should contain the pool of images used to create the mosaic. Each image in this folder will be considered as a potential part of the mosaic. Ensure that this folder has enough images to cover the entire mosaic; otherwise, the script will terminate with an error indicating insufficient images. Update the `pool_path` in the `Options` class to change the folder path.

### Running the Script
After setting up your environment and ensuring all paths are correctly set in the `Options` class, run the script from the command line or a Python environment. The script will process the images and save the resulting mosaic to the specified output path.

### Notes
- The script includes a progress bar via `tqdm` to indicate the progress of the mosaic generation.
- Adjustments may be necessary for the `stride` parameter in the `Options` class depending on the size of the input images and the desired granularity of the mosaic.

In [13]:
import cv2
import numpy as np
import os
from itertools import product
from tqdm import tqdm

class Options:
    def __init__(self, input_path="data/input.jpg", output_path="data/output.jpg", pool_path="jds", stride=35):
        self.input = input_path
        self.output = output_path
        self.pool = pool_path
        self.stride = stride

def get_component_images(path, size):
    images = []
    avg_colors = []
    all_files = os.listdir(path)
    image_paths = [os.path.join(path, f) for f in all_files if os.path.isfile(os.path.join(path, f))]
    for image_path in tqdm(image_paths, desc="Reading images"):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        if image is not None:
            image = cv2.resize(image, (size, size))
            images.append(image)
            avg_colors.append(np.mean(image, axis=(0, 1)))
    return images, np.array(avg_colors)

def calculate_max_dimensions(opt, input_image):
    input_height, input_width, _ = input_image.shape
    max_width = input_width // opt.stride * opt.stride
    max_height = input_height // opt.stride * opt.stride
    return max_width, max_height

def calculate_required_images(opt, input_image):
    max_width, max_height = calculate_max_dimensions(opt, input_image)
    num_horizontal_tiles = max_width // opt.stride
    num_vertical_tiles = max_height // opt.stride
    return num_horizontal_tiles * num_vertical_tiles

def main(opt):
    try:
        os.makedirs("output_snapshots", exist_ok=True)
        input_image = cv2.imread(opt.input, cv2.IMREAD_COLOR)
        if input_image is None:
            raise ValueError(f"Failed to load input image from {opt.input}")

        required_images = calculate_required_images(opt, input_image)
        available_images = len([f for f in os.listdir(opt.pool) if os.path.isfile(os.path.join(opt.pool, f))])

        if available_images < required_images:
            print(f"Insufficient images to create the mosaic. Required: {required_images}, Available: {available_images}")
            return

        max_width, max_height = calculate_max_dimensions(opt, input_image)
        blank_image = np.zeros((max_height, max_width, 3), np.uint8)
        images, avg_colors = get_component_images(opt.pool, opt.stride)

        used_indices = set()
        total_steps = required_images
        progress_bar = tqdm(total=total_steps, desc="Generating mosaic")

        for i, j in product(range(0, max_width, opt.stride), range(0, max_height, opt.stride)):
            target_section = input_image[j:j+opt.stride, i:i+opt.stride]
            # Adjust the darkness threshold or remove this check as needed
            if np.mean(target_section) < 30:
                progress_bar.update(1)
                continue

            best_match_idx = None
            min_dist = float('inf')
            target_color = np.mean(target_section, axis=(0, 1))

            for idx, color in enumerate(avg_colors):
                if idx not in used_indices:
                    dist = np.linalg.norm(color - target_color)
                    if dist < min_dist:
                        min_dist = dist
                        best_match_idx = idx

            if best_match_idx is not None:
                blank_image[j:j+opt.stride, i:i+opt.stride] = images[best_match_idx]
                used_indices.add(best_match_idx)

            progress_bar.update(1)

        cv2.imwrite(opt.output, blank_image)
        print(f"Final image successfully saved to {opt.output}")
    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        if 'progress_bar' in locals():
            progress_bar.close()

# Update these paths and parameters as needed
opt = Options(input_path="data/input.jpg", output_path="data/output.jpg", pool_path="jds", stride=35)
main(opt)

Generating mosaic: 100%|████████████████████| 598/598 [00:00<00:00, 2986.30it/s]

Final image successfully saved to data/output.jpg
